In [1]:
import pandas as pd

from openff.evaluator.datasets import (
    PhysicalPropertyDataSet,
)

from openff.evaluator.client import RequestOptions
from openff.evaluator import unit
from openff.evaluator.storage import LocalFileStorage
from openff.evaluator.workflow import Workflow

def dataset_from_csv(csv="01_mnsol_data.csv"):
    df = pd.read_csv(csv)
    df = df.iloc[:1]
    dataset = PhysicalPropertyDataSet.from_pandas(df)
    return dataset

def entry_from_csv(csv="01_mnsol_data.csv", entry=0):
    df = pd.read_csv(csv)
    dataset = PhysicalPropertyDataSet.from_pandas(df)
    return dataset.properties[entry]

def estimate_entry(entry, ff, working_directory=""):
    from openff.evaluator.forcefield import SmirnoffForceFieldSource
    property_type = type(entry)
    
    # Set up schema. Here I know the force field is SMIRNOFF
    schema = property_type.default_simulation_schema().workflow_schema
    schema.replace_protocol_types({"BaseBuildSystem": "BuildSmirnoffSystem"})
    
    # force field
    FF_FILE = "force-field.json"
    ff_source = SmirnoffForceFieldSource.from_object(ff)
    ff_source.json(FF_FILE)
    
    metadata = Workflow.generate_default_metadata(entry, FF_FILE, [])
    workflow = Workflow.from_schema(schema, metadata=metadata)
    return workflow
    result = workflow.execute(root_directory=working_directory)

In [2]:
from openff.toolkit.typing.engines.smirnoff import ForceField


ff = ForceField("openff_unconstrained-1.3.0.offxml")

/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)


In [10]:
entry.substance.components[0].smiles

'Cc1ccccn1'

In [3]:
entry = entry_from_csv()

/Users/lily/anaconda3/envs/polymetrizer/lib/python3.9/site-packages/cmiles/_cmiles_rd.py:349: UserWarning: Stereochemistry is unspecified. Problematic atoms [(4, 'C')], problematic bonds []
  warnings.warn("Stereochemistry is unspecified. Problematic atoms {}, problematic bonds {}".format(
/Users/lily/anaconda3/envs/polymetrizer/lib/python3.9/site-packages/cmiles/_cmiles_rd.py:349: UserWarning: Stereochemistry is unspecified. Problematic atoms [(1, 'C')], problematic bonds []
  warnings.warn("Stereochemistry is unspecified. Problematic atoms {}, problematic bonds {}".format(
RDKit WARNING: [15:50:22] WARNING: not removing hydrogen atom without neighbors
/Users/lily/anaconda3/envs/polymetrizer/lib/python3.9/site-packages/cmiles/_cmiles_rd.py:349: UserWarning: Stereochemistry is unspecified. Problematic atoms [(2, 'C')], problematic bonds []
  warnings.warn("Stereochemistry is unspecified. Problematic atoms {}, problematic bonds {}".format(
[15:50:22] WARNING: not removing hydrogen atom 

In [4]:
wk = estimate_entry(entry, ff)

In [5]:
result = wk.execute()

KeyboardInterrupt: 

In [ ]:
for x in result.exceptions[0].message:
    print(x)

In [ ]:
result.value

In [13]:
wk.

openff.evaluator.workflow.workflow.Workflow

In [93]:

from openff.evaluator.properties import Density, EnthalpyOfVaporization, SolvationFreeEnergy
from openff.evaluator.client import RequestOptions

# Create an options object which defines how the data set should be estimated.
estimation_options = RequestOptions()
# Specify that we only wish to use molecular simulation to estimate the data set.
estimation_options.calculation_layers = ["SimulationLayer"]

estimation_options.add_schema("SimulationLayer", "SolvationFreeEnergy", SolvationFreeEnergy.default_simulation_schema())

In [94]:
from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster

calculation_backend = DaskLocalCluster(
    number_of_workers=1,
    resources_per_worker=ComputeResources(
        number_of_threads=1,
#         number_of_gpus=1,
#         preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA
    ),
)
calculation_backend.start()



/Users/lily/anaconda3/envs/polymetrizer/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55460 instead
  warnings.warn(


In [95]:
from openff.evaluator.server import EvaluatorServer

evaluator_server = EvaluatorServer(calculation_backend=calculation_backend)
evaluator_server.start(asynchronous=True)

OSError: [Errno 48] Address already in use

In [1]:
data_set = dataset_from_csv()

NameError: name 'dataset_from_csv' is not defined

In [86]:
from openff.evaluator.forcefield import SmirnoffForceFieldSource

force_field_path = "openff-1.0.0.offxml"
force_field_source = SmirnoffForceFieldSource.from_path(force_field_path)

/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)


In [87]:
from openff.evaluator.client import EvaluatorClient
evaluator_client = EvaluatorClient()

request, exception = evaluator_client.request_estimate(
    property_set=data_set,
    force_field_source=force_field_source,
    options=estimation_options,
)

assert exception is None

/Users/lily/anaconda3/envs/polymetrizer/lib/python3.9/site-packages/cmiles/_cmiles_rd.py:349: UserWarning: Stereochemistry is unspecified. Problematic atoms [(4, 'C')], problematic bonds []
  warnings.warn("Stereochemistry is unspecified. Problematic atoms {}, problematic bonds {}".format(
/Users/lily/anaconda3/envs/polymetrizer/lib/python3.9/site-packages/cmiles/_cmiles_rd.py:349: UserWarning: Stereochemistry is unspecified. Problematic atoms [(1, 'C')], problematic bonds []
  warnings.warn("Stereochemistry is unspecified. Problematic atoms {}, problematic bonds {}".format(
RDKit WARNING: [19:36:41] WARNING: not removing hydrogen atom without neighbors
[19:36:41] WARNING: not removing hydrogen atom without neighbors
/Users/lily/anaconda3/envs/polymetrizer/lib/python3.9/site-packages/cmiles/_cmiles_rd.py:349: UserWarning: Stereochemistry is unspecified. Problematic atoms [(2, 'C')], problematic bonds []
  warnings.warn("Stereochemistry is unspecified. Problematic atoms {}, problematic 

In [78]:
result = wk.execute()

In [89]:
results, exception = request.results(synchronous=True, polling_interval=30)
assert exception is None

RDKit WARNING: [20:02:07] WARNING: not removing hydrogen atom without neighbors
[20:02:07] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:02:07] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:02:07] WARNING: not removing hydrogen atom without neighbors
[20:02:07] WARNING: not removing hydrogen atom without neighbors
[20:02:07] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:02:07] WARNING: not removing hydrogen atom without neighbors
[20:02:07] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:02:07] WARNING: not removing hydrogen atom without neighbors
[20:02:07] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:02:07] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:02:07] WARNING: not removing hydrogen atom without neighbors
[20:02:07] WARNING: not removing hydrogen atom without neighbors
[20:02:07] WARNING: not removing hydrogen atom wit

/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/anaconda3/envs/polymetrizer/lib/python3.9/site-packages/distributed/worker.py:3862: UserWarning: Large object of size 32.69 MiB detected in task graph: 
  [<function WorkflowCalculationLayer.workflow_to_la ... c871f1e344c9']]
Consider scattering large objects ahead of time
with client.scatter to redu

/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
RDKit WARNING: [20:11:43] WARNING: not removing hydrogen atom without neighbors
[20:11:43] WARNING: not removing hydrogen atom without neighbors
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the Open

/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
RDKit WARNING: [20:14:07] WARNING: not removing hydrogen atom without neighbors
[20:14:07] WARNING: not removing hydrogen atom without neighbors
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the Open

/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
RDKit WARNING: [20:18:19] WARNING: not removing hydrogen atom without neighbors
[20:

...

RDKit WARNING: [21:27:17] WARNING: not removing hydrogen atom without neighbors
[21:27:17] WARNING: not removing hydrogen atom without neighbors
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
RDKit WARNING: [21:27:22] WARNING: not removing hydrogen atom without neighbors
[21:27:22] WARNING: not removing hydrogen atom without neighbors
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openf

/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationW

/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationW

RDKit WARNING: [21:34:06] WARNING: not removing hydrogen atom without neighbors
[21:34:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [21:34:10] WARNING: not removing hydrogen atom without neighbors
[21:34:10] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [21:34:15] WARNING: not removing hydrogen atom without neighbors
[21:34:15] WARNING: not removing hydrogen atom without neighbors
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is DEPRECATED and will be removed in a future release of the OpenFF Toolkit.
  warnings.warn(warning_msg, DeprecationWarning)
/Users/lily/pydev/openff-toolkit/openff/toolkit/utils/exceptions.py:18: DeprecationWarning: ParseError is D

KeyboardInterrupt: 

In [74]:
result.data_to_store

[]

In [51]:
dir(wk)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_append_uuid_to_output_to_store',
 '_apply_replicator',
 '_apply_replicator_to_outputs',
 '_apply_replicator_to_replicators',
 '_apply_replicators',
 '_build_output_to_store',
 '_build_protocols',
 '_final_value_source',
 '_find_relevant_gradient_keys',
 '_get_schema',
 '_get_template_values',
 '_global_metadata',
 '_outputs_to_store',
 '_protocols',
 '_set_schema',
 'execute',
 'final_value_source',
 'from_schema',
 'generate_default_metadata',
 'outputs_to_store',
 'protocols',
 'replace_protocol',
 'schema',
 'to_graph',
 'uuid']

In [58]:
entry.json()

'{"id": "bc090eb3ab4e4a5e825eb63cf7b4e00a", "substance": {"components": [{"smiles": "Cc1ccccn1", "role": {"value": "solv", "@type": "openff.evaluator.substances.components.Component.Role"}, "@type": "openff.evaluator.substances.components.Component"}, {"smiles": "CCCCCCCC", "role": {"value": "sol", "@type": "openff.evaluator.substances.components.Component.Role"}, "@type": "openff.evaluator.substances.components.Component"}], "amounts": {"Cc1ccccn1{solv}": [{"value": 1.0, "@type": "openff.evaluator.substances.amounts.MoleFraction"}], "CCCCCCCC{sol}": [{"value": 1, "@type": "openff.evaluator.substances.amounts.ExactAmount"}]}, "@type": "openff.evaluator.substances.substances.Substance"}, "phase": 2, "thermodynamic_state": {"temperature": {"value": 298.0, "unit": "K", "@type": "openff.evaluator.unit.Quantity"}, "pressure": {"value": 101.325, "unit": "kPa", "@type": "openff.evaluator.unit.Quantity"}, "@type": "openff.evaluator.thermodynamics.ThermodynamicState"}, "value": {"value": -19.79

In [63]:
Workflow({}).generate_default_metadata(entry, "openff-1.0.0.offxml", parameter_gradient_keys=[])

{'thermodynamic_state': <ThermodynamicState T=298.0 K P=101.325 kPa>,
 'substance': <Substance CCCCCCCC{sol}{n=1}|Cc1ccccn1{solv}{x=1.000000}>,
 'components': [<Substance Cc1ccccn1{solv}{x=1.000000}>,
  <Substance CCCCCCCC{sol}{x=1.000000}>],
 'target_uncertainty': inf <Unit('kilojoule / mole')>,
 'per_component_uncertainty': inf <Unit('kilojoule / mole')>,
 'force_field_path': 'openff-1.0.0.offxml',
 'parameter_gradient_keys': []}

In [34]:
dir(entry)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_gradients',
 '_id',
 '_metadata',
 '_phase',
 '_set_value',
 '_source',
 '_substance',
 '_thermodynamic_state',
 '_uncertainty',
 '_value',
 'default_simulation_schema',
 'default_unit',
 'from_json',
 'get_attributes',
 'gradients',
 'id',
 'json',
 'metadata',
 'parse_json',
 'phase',
 'source',
 'substance',
 'thermodynamic_state',
 'uncertainty',
 'validate',
 'value']

In [16]:
schema = entry.default_simulation_schema()

In [29]:
schema.workflow_schema

In [22]:
schema.workflow_schema.json()

'{"protocol_schemas": [{"id": "build_solvated_coordinates", "type": "BuildCoordinatesPackmol", "inputs": {".allow_merging": true, ".max_molecules": 2000, ".count_exact_amount": true, ".mass_density": {"value": 0.95, "unit": "g / ml", "@type": "openff.evaluator.unit.Quantity"}, ".box_aspect_ratio": [1.0, 1.0, 1.0], ".substance": {"full_path": "global.substance", "@type": "openff.evaluator.workflow.utils.ProtocolPath"}, ".tolerance": {"value": 2.0, "unit": "\\u00c5", "@type": "openff.evaluator.unit.Quantity"}, ".verbose_packmol": false, ".retain_packmol_files": false}, "@type": "openff.evaluator.workflow.schemas.ProtocolSchema"}, {"id": "assign_solvated_parameters", "type": "BaseBuildSystem", "inputs": {".allow_merging": true, ".force_field_path": {"full_path": "global.force_field_path", "@type": "openff.evaluator.workflow.utils.ProtocolPath"}, ".coordinate_file_path": {"full_path": "build_solvated_coordinates.coordinate_file_path", "@type": "openff.evaluator.workflow.utils.ProtocolPath"

In [4]:
from openff.evaluator.datasets import (
    PhysicalPropertyDataSet,
)

In [5]:
ds = PhysicalPropertyDataSet()

In [10]:
dir(ds)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_properties',
 'add_properties',
 'from_json',
 'from_pandas',
 'json',
 'merge',
 'parse_json',
 'properties',
 'properties_by_substance',
 'properties_by_type',
 'property_types',
 'sources',
 'substances',
 'to_pandas',
 'validate']